In [1]:
#Importing the Libraries
import time
import psutil, os

#Expanded nodes count the exploration of nodes during the Search
global expanded_nodes
expanded_nodes = 0

#goal_state denotes the final goal state that is searched for
global goal_state
goal_state = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "0"]

#Used to backtrack the Initial node, The Operations Performed to Reach Final node
global path
path = []

#Environment function describes the state of the Environment and Possible Actions on the Environment
class Environment:
    def __init__(self,board):
        self.side = 4
        self.board = board
    #This function returns the outputing state from taking particular action from current state
    def perform_action(self,action):
        new_board = self.board[:]
        zero_index = new_board.index('0')
        
        if action == "L" and (zero_index % 4)> 0:
            temp_L = new_board[zero_index - 1]
            new_board[zero_index - 1] = new_board[zero_index]
            new_board[zero_index] = temp_L
            
        if action == "R" and (zero_index % 4)< 3:
            temp_R = new_board[zero_index + 1]
            new_board[zero_index + 1] = new_board[zero_index]
            new_board[zero_index] = temp_R
            
        if action == "U" and (zero_index / 4)>= 1:
            temp_U = new_board[zero_index - 4]
            new_board[zero_index - 4] = new_board[zero_index]
            new_board[zero_index] = temp_U
            
        if action == "D" and (zero_index / 4)< 3:
            temp_D = new_board[zero_index + 4]
            new_board[zero_index + 4] = new_board[zero_index]
            new_board[zero_index] = temp_D
            
        return Environment(new_board)

#Class to initialize State of Node, Parent of Node, and Action Performed
class Node:
    def __init__(self,state,parent,action):
        self.state = state
        self.parent = parent
        self.action = action
        #A Node might not have parent if it has not been explored yet, or if it's the Initial node
        if parent:
            self.depth = parent.depth + 1
        elif parent is None:
            self.depth = 0
            
    def __repr__(self):
        return str(self.state.board)
    
    #Used to compare multiple boards
    def __eq__(self,other):
        return self.state.board == other.state.board

#Function that performs Iterative Deepening Depth-First Search Algorithm
#Inputs are taken as Current Node and Varying Values of Depth (Hard-limited at 100)
def IDDFS(current_node,depth):
    global expanded_nodes
    #Checking if Current Node is same as Goal
    if current_node.state.board == goal_state:
        return current_node
    elif depth == 0: 
        return 0
    #If the Current Node is not at Depth 0 and if the Goal has not been spotted yet, we explore children nodes
    else:
        flag = 0
        #Increment Number of Nodes Expanded
        expanded_nodes = expanded_nodes + 1
        children = []
        #List of Possible Actions that can be performed
        actions = ["U", "R", "D", "L"]
        for action in actions:
            #State after performing Action
            child_state = current_node.state.perform_action(action)
            #Creating Node with Parent Node, and Action performed on Current Node
            child_node = Node(child_state, current_node,action)
            children.append(child_node)
            #In Depth-First Search, we explore the Children Node before moving on
            output = IDDFS(child_node, depth - 1)
            #Checking for Flags
            type_ = type(output)
            if type_ == int:
                if output == 0:
                    flag = 1
            else:
                return output
        if flag == 1:
            return 0
        else: 
            return -1

#Main function
def main():
    #This is used to monitor the Memory usage of the Algorithm
    process = psutil.Process(os.getpid())
    initial_memory = process.memory_info().rss / 1024.0
    
    #Input is obtained from the User, and is taken as a String
    initial_state = str(input("Enter the Initial Configuration: "))
    
    #Performing Initial Checks
    if len(initial_state) < 37:
        print("Insufficient Data. Please Try Again")
        time.sleep(5)
        exit()
        
    #It was found out that this Combination cannot be solved
    if initial_state == "1 2 3 4 5 6 7 8 9 10 11 12 13 15 14 0":
        print("Impossible to Compute this Input Combination")
        time.sleep(5)
        exit()
    
    #Splitting the User Input based on Whitespace
    initial_state = initial_state.split(" ")
    #Creating Initial node with Parent Node and Action set as None
    initial_node = Node(Environment(initial_state),None,None)
    
    global expanded_nodes
    #Beginning IDDFS
    #time.time() function is used to obtain instantaneous time
    commencement = time.time()
    #Hard-limiting the code to a Max Depth of 100
    for depth in range(0,100):
        expanded_nodes = 0
        print("Depth: ", depth)
        output = IDDFS(initial_node, depth)
        #Analyzing the Type of output obtained based on Return Statements in IDDFS function
        type_ = type(output)
        #If the Goal Node was spotted, we backtrack to Initial Node to obtain path taken to get there
        if type_ == Node:
            while(output.parent is not None):
                path.append(output.action)
                output = output.parent
            path.reverse()
            break
        #This condition denotes failure in reaching the goal at that particular depth
        elif output == 0:
            print("No Solution at this Depth: ", depth)
        #This condition denotes failure to reach goal after all depths
        else:
            print("Error 404! Solution not Found!")
    conclusion = time.time()
    final_memory = process.memory_info().rss / 1024.0
    #Function to Print final details of operation
    final_print(commencement, conclusion, initial_memory, final_memory, path, expanded_nodes)
    
#Function to Print final details of operation 
def final_print(commencement, conclusion, initial_memory, final_memory, path, expanded_nodes):
    print("Moves: ",path)
    print("Number of Nodes expanded: ",expanded_nodes)
    elapsed_time = conclusion - commencement
    print("Time Taken: ",str(elapsed_time))
    memory_usage = final_memory - initial_memory
    print("Memory Used: " +str(memory_usage)+ " kb")
    

if __name__ == "__main__":
    main()

Enter the Initial Configuration: 1 0 2 4 5 7 3 8 9 6 11 12 13 10 14 15
Depth:  0
No Solution at this Depth:  0
Depth:  1
No Solution at this Depth:  1
Depth:  2
No Solution at this Depth:  2
Depth:  3
No Solution at this Depth:  3
Depth:  4
No Solution at this Depth:  4
Depth:  5
No Solution at this Depth:  5
Depth:  6
No Solution at this Depth:  6
Depth:  7
Moves:  ['R', 'D', 'L', 'D', 'D', 'R', 'R']
Number of Nodes expanded:  2362
Time Taken:  0.04387950897216797
Memory Used: 224.0 kb
